# ACLED Dataset EDA - part 2

In [36]:
#this notebook is used for preprocessing the notes column

import pandas as pd
import os
import csv
import requests
import re 
engine='python'


import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import seaborn as sns



import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk import sent_tokenize, word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ntsegie\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ntsegie\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ntsegie\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [37]:
df = pd.read_csv('2019-05-01-2021-05-29-Ethiopia.csv', encoding = 'utf-8')

df.head(5)

,data_id,iso,event_id_cnty,event_id_no_cnty,event_date,year,time_precision,event_type,sub_event_type,actor1,...,location,latitude,longitude,geo_precision,source,source_scale,notes,fatalities,timestamp,iso3
0,7993281,231,ETH6788,6788,21 May 2021,2021,1,Violence against civilians,Attack,Police Forces of Ethiopia (2018-) Oromo Region...,...,Jarso,9.1097,37.3727,1,Oromiya Media Network,International,"On 21 May 2021, Oromo regional special forces ...",1,1621889045,ETH
1,7992817,231,ETH6787,6787,21 May 2021,2021,1,Protests,Peaceful protest,Protesters (Ethiopia),...,Addis Ababa,9.0250,38.7469,1,BBC News,International,"On 21 May 2021, protesters gathered in Addis A...",0,1621889044,ETH
2,7993003,231,ETH6785,6785,20 May 2021,2021,1,Protests,Excessive force against protesters,Protesters (Ethiopia),...,Mer Awi,11.4167,37.1667,2,Deutsche Welle,International,"On 20 May 2021, police forces clashed with dem...",4,1621889044,ETH
3,7993825,231,ETH6786,6786,20 May 2021,2021,1,Battles,Armed clash,Afar Ethnic Militia (Ethiopia),...,Shinile,9.6881,41.8447,2,Voss TV,Subnational,"On 20 May 2021, Afar militias attacked a Somal...",6,1621889046,ETH
4,7993278,231,ETH6784,6784,19 May 2021,2021,1,Violence against civilians,Attack,Military Forces of Ethiopia (2018-),...,Senbete,10.3069,39.9885,2,Oromiya Media Network,International,"On 19 May 2021, military forces in Jille Dumug...",1,1621889045,ETH


In [38]:
#converting text to lower case
df['notes'] = df['notes'].str.lower()

In [39]:
example_notes = df.iloc[0]
print(example_notes['notes'])

on 21 may 2021, oromo regional special forces shot and killed an 11th grade student in jarso woreda (west wellega, oromia). no further information.


In [40]:
#tokenization

print (nltk.word_tokenize(example_notes['notes']))

['on', '21', 'may', '2021', ',', 'oromo', 'regional', 'special', 'forces', 'shot', 'and', 'killed', 'an', '11th', 'grade', 'student', 'in', 'jarso', 'woreda', '(', 'west', 'wellega', ',', 'oromia', ')', '.', 'no', 'further', 'information', '.']


In [19]:
#word tokenizing

def identify_tokens(row):
    notes = row['notes']
    tokens = nltk.word_tokenize(notes)
    # taken only words (not punctuation)
    token_words = [w for w in tokens if w.isalpha()]
    return token_words

df['token_words'] = df.apply(identify_tokens, axis=1)
df.head(5)

,data_id,iso,event_id_cnty,event_id_no_cnty,event_date,year,time_precision,event_type,sub_event_type,actor1,...,longitude,geo_precision,source,source_scale,notes,fatalities,timestamp,iso3,words,token_words
0,7993281,231,ETH6788,6788,21 May 2021,2021,1,Violence against civilians,Attack,Police Forces of Ethiopia (2018-) Oromo Region...,...,37.3727,1,Oromiya Media Network,International,"on 21 may 2021, oromo regional special forces ...",1,1621889045,ETH,"[on, may, oromo, regional, special, forces, sh...","[on, may, oromo, regional, special, forces, sh..."
1,7992817,231,ETH6787,6787,21 May 2021,2021,1,Protests,Peaceful protest,Protesters (Ethiopia),...,38.7469,1,BBC News,International,"on 21 may 2021, protesters gathered in addis a...",0,1621889044,ETH,"[on, may, protesters, gathered, in, addis, aba...","[on, may, protesters, gathered, in, addis, aba..."
2,7993003,231,ETH6785,6785,20 May 2021,2021,1,Protests,Excessive force against protesters,Protesters (Ethiopia),...,37.1667,2,Deutsche Welle,International,"on 20 may 2021, police forces clashed with dem...",4,1621889044,ETH,"[on, may, police, forces, clashed, with, demon...","[on, may, police, forces, clashed, with, demon..."
3,7993825,231,ETH6786,6786,20 May 2021,2021,1,Battles,Armed clash,Afar Ethnic Militia (Ethiopia),...,41.8447,2,Voss TV,Subnational,"on 20 may 2021, afar militias attacked a somal...",6,1621889046,ETH,"[on, may, afar, militias, attacked, a, somali,...","[on, may, afar, militias, attacked, a, somali,..."
4,7993278,231,ETH6784,6784,19 May 2021,2021,1,Violence against civilians,Attack,Military Forces of Ethiopia (2018-),...,39.9885,2,Oromiya Media Network,International,"on 19 may 2021, military forces in jille dumug...",1,1621889045,ETH,"[on, may, military, forces, in, jille, dumuga,...","[on, may, military, forces, in, jille, dumuga,..."


In [41]:
#stemming
from nltk.stem import PorterStemmer
stemming = PorterStemmer()
def stem_list(row):
    my_list = row['notes']
    stemmed_list = [stemming.stem(word) for word in my_list]
    return (stemmed_list)

df['stemmed_words'] = df.apply(stem_list, axis=1)

df.head(5)

,data_id,iso,event_id_cnty,event_id_no_cnty,event_date,year,time_precision,event_type,sub_event_type,actor1,...,latitude,longitude,geo_precision,source,source_scale,notes,fatalities,timestamp,iso3,stemmed_words
0,7993281,231,ETH6788,6788,21 May 2021,2021,1,Violence against civilians,Attack,Police Forces of Ethiopia (2018-) Oromo Region...,...,9.1097,37.3727,1,Oromiya Media Network,International,"on 21 may 2021, oromo regional special forces ...",1,1621889045,ETH,"[o, n, , 2, 1, , m, a, y, , 2, 0, 2, 1, ,, ..."
1,7992817,231,ETH6787,6787,21 May 2021,2021,1,Protests,Peaceful protest,Protesters (Ethiopia),...,9.0250,38.7469,1,BBC News,International,"on 21 may 2021, protesters gathered in addis a...",0,1621889044,ETH,"[o, n, , 2, 1, , m, a, y, , 2, 0, 2, 1, ,, ..."
2,7993003,231,ETH6785,6785,20 May 2021,2021,1,Protests,Excessive force against protesters,Protesters (Ethiopia),...,11.4167,37.1667,2,Deutsche Welle,International,"on 20 may 2021, police forces clashed with dem...",4,1621889044,ETH,"[o, n, , 2, 0, , m, a, y, , 2, 0, 2, 1, ,, ..."
3,7993825,231,ETH6786,6786,20 May 2021,2021,1,Battles,Armed clash,Afar Ethnic Militia (Ethiopia),...,9.6881,41.8447,2,Voss TV,Subnational,"on 20 may 2021, afar militias attacked a somal...",6,1621889046,ETH,"[o, n, , 2, 0, , m, a, y, , 2, 0, 2, 1, ,, ..."
4,7993278,231,ETH6784,6784,19 May 2021,2021,1,Violence against civilians,Attack,Military Forces of Ethiopia (2018-),...,10.3069,39.9885,2,Oromiya Media Network,International,"on 19 may 2021, military forces in jille dumug...",1,1621889045,ETH,"[o, n, , 1, 9, , m, a, y, , 2, 0, 2, 1, ,, ..."


In [42]:
#stopwords

from nltk.corpus import stopwords
stops = set(stopwords.words("english"))                  

def remove_stops(row):
    my_list = row['stemmed_words']
    meaningful_words = [w for w in my_list if not w in stops]
    return (meaningful_words)

df['stem_meaningful'] = df.apply(remove_stops, axis=1)

print(df)

      data_id  iso event_id_cnty  event_id_no_cnty   event_date  year  \
0     7993281  231       ETH6788              6788  21 May 2021  2021   
1     7992817  231       ETH6787              6787  21 May 2021  2021   
2     7993003  231       ETH6785              6785  20 May 2021  2021   
3     7993825  231       ETH6786              6786  20 May 2021  2021   
4     7993278  231       ETH6784              6784  19 May 2021  2021   
...       ...  ...           ...               ...          ...   ...   
1580  7675911  231       ETH5183              5183  02 May 2019  2019   
1581  7675181  231       ETH5248              5248  02 May 2019  2019   
1582  7771530  231       ETH5247              5247  02 May 2019  2019   
1583  7771533  231       ETH5246              5246  02 May 2019  2019   
1584  7675892  231       ETH5182              5182  01 May 2019  2019   

      time_precision                  event_type  \
0                  1  Violence against civilians   
1                  

In [43]:
#save our processed data
cols_to_drop = ['event_type', 'notes', 'words', 'stemmed_words', 'stem_meaningful']
df.drop(cols_to_drop, inplace=True)

df.to_csv('df_processed.csv', index=False)

KeyError: "['event_type' 'notes' 'words' 'stemmed_words' 'stem_meaningful'] not found in axis"

,data_id,iso,event_id_cnty,event_id_no_cnty,event_date,year,time_precision,event_type,sub_event_type,actor1,assoc_actor_1,inter1,actor2,assoc_actor_2,inter2,interaction,region,country,admin1,admin2,admin3,location,latitude,longitude,geo_precision,source,source_scale,notes,fatalities,timestamp,iso3
0,7993281,231,ETH6788,6788,21 May 2021,2021,1,Violence against civilians,Attack,Police Forces of Ethiopia (2018-) Oromo Region...,NaN,1,Civilians (Ethiopia),Students (Ethiopia),7,17,Eastern Africa,Ethiopia,Oromia,West Shewa,Cheliya,Jarso,9.1097,37.3727,1,Oromiya Media Network,International,"On 21 May 2021, Oromo regional special forces ...",1,1621889045,ETH
1,7992817,231,ETH6787,6787,21 May 2021,2021,1,Protests,Peaceful protest,Protesters (Ethiopia),NaN,6,NaN,NaN,0,60,Eastern Africa,Ethiopia,Addis Ababa,Region 14,Lideta,Addis Ababa,9.0250,38.7469,1,BBC News,International,"On 21 May 2021, protesters gathered in Addis A...",0,1621889044,ETH
2,7993003,231,ETH6785,6785,20 May 2021,2021,1,Protests,Excessive force against protesters,Protesters (Ethiopia),Students (Ethiopia),6,Police Forces of Ethiopia (2018-),NaN,1,16,Eastern Africa,Ethiopia,Amhara,West Gojam,Marawi Town,Mer Awi,11.4167,37.1667,2,Deutsche Welle,International,"On 20 May 2021, police forces clashed with dem...",4,1621889044,ETH
3,7993825,231,ETH6786,6786,20 May 2021,2021,1,Battles,Armed clash,Afar Ethnic Militia (Ethiopia),NaN,4,Somali Ethnic Militia (Ethiopia),NaN,4,44,Eastern Africa,Ethiopia,Somali,Siti,Shinile,Shinile,9.6881,41.8447,2,Voss TV,Subnational,"On 20 May 2021, Afar militias attacked a Somal...",6,1621889046,ETH
4,7993278,231,ETH6784,6784,19 May 2021,2021,1,Violence against civilians,Attack,Military Forces of Ethiopia (2018-),NaN,1,Civilians (Ethiopia),Pastoralists (Ethiopia),7,17,Eastern Africa,Ethiopia,Amhara,Oromia,Jilye Tumuga,Senbete,10.3069,39.9885,2,Oromiya Media Network,International,"On 19 May 2021, military forces in Jille Dumug...",1,1621889045,ETH


In [35]:
#finding the TF

def tf(corpus):
    dic={}
    for df in corpus:
        for word in df.split():
            if word in dic:
                dic[word] = dic[word] + 1
            else:
                dic[word]=1
    for word,freq in dic.items():
        print(word,freq)
        dic[word]=freq/sum(map(len, (df.split() for df in corpus)))
    return dic
tf(df)

data_id 1
iso 1
event_id_cnty 1
event_id_no_cnty 1
event_date 1
year 1
time_precision 1
event_type 1
sub_event_type 1
actor1 1
assoc_actor_1 1
inter1 1
actor2 1
assoc_actor_2 1
inter2 1
interaction 1
region 1
country 1
admin1 1
admin2 1
admin3 1
location 1
latitude 1
longitude 1
geo_precision 1
source 1
source_scale 1
notes 1
fatalities 1
timestamp 1
iso3 1
words 1
token_words 1
stemmed_words 1


{'data_id': 0.029411764705882353,
 'iso': 0.029411764705882353,
 'event_id_cnty': 0.029411764705882353,
 'event_id_no_cnty': 0.029411764705882353,
 'event_date': 0.029411764705882353,
 'year': 0.029411764705882353,
 'time_precision': 0.029411764705882353,
 'event_type': 0.029411764705882353,
 'sub_event_type': 0.029411764705882353,
 'actor1': 0.029411764705882353,
 'assoc_actor_1': 0.029411764705882353,
 'inter1': 0.029411764705882353,
 'actor2': 0.029411764705882353,
 'assoc_actor_2': 0.029411764705882353,
 'inter2': 0.029411764705882353,
 'interaction': 0.029411764705882353,
 'region': 0.029411764705882353,
 'country': 0.029411764705882353,
 'admin1': 0.029411764705882353,
 'admin2': 0.029411764705882353,
 'admin3': 0.029411764705882353,
 'location': 0.029411764705882353,
 'latitude': 0.029411764705882353,
 'longitude': 0.029411764705882353,
 'geo_precision': 0.029411764705882353,
 'source': 0.029411764705882353,
 'source_scale': 0.029411764705882353,
 'notes': 0.029411764705882353,
